In [110]:
import pandas as pd
import numpy as np

In [111]:
MISSING_UBIGEO = [
    {
        'department_name': 'Loreto',
        'province_name': 'Alto Amazonas',
        'district_name': 'Barranca',
        'department_id': '16',
        'province_id': '1602',
        'district_id': '160203'
    },
    {
        'department_name': 'Loreto',
        'province_name': 'Alto Amazonas',
        'district_name': 'Manseriche',
        'department_id': '16',
        'province_id': '1602',
        'district_id': '160207'
    },
    {
        'department_name': 'Loreto',
        'province_name': 'Alto Amazonas',
        'district_name': 'Morona',
        'department_id': '16',
        'province_id': '1602',
        'district_id': '160208'
    },
    {
        'department_name': 'Loreto',
        'province_name': 'Alto Amazonas',
        'district_name': 'Pastaza',
        'department_id': '16',
        'province_id': '1602',
        'district_id': '160209'
    },
    {
        'department_name': 'Otro',
        'province_name': 'Otro',
        'district_name': 'Otro',
        'department_id': '99',
        'province_id': '9999',
        'district_id': '999999'
    }
]

In [112]:
        df = pd.read_excel('../../../datasets/anexos/2.Ubigeo_descripción.xlsx', header=2, usecols='B,E,F')
        
        df.replace(' ', np.nan, inplace=True)
        df.dropna(inplace=True)
        df = df[df['DEPARTAMENTO'] != 'DEPARTAMENTO'].copy()

        df.rename(columns={
            'DEPARTAMENTO': 'department_name',
            'PROVINCIA': 'province_name',
            'DISTRITO': 'district_name'
        }, inplace=True)

        columns = df.columns

        for column in columns:
            df['{}_id'.format(column.split('_')[0])] = df[column].astype(str).str[0:2]
            df[column] = df[column].astype(str).str[3:]

        df['province_id'] = df['department_id'] + df['province_id']
        df['district_id'] = df['province_id'] + df['district_id']

        df['department_name'] = df['department_name'].str.strip()
        df['province_name'] = df['province_name'].str.strip()
        df['district_name'] = df['district_name'].str.strip()

        df = df.append(MISSING_UBIGEO)

        provinces = df[['department_name', 'province_name', 'department_id', 'province_id']].copy()
        provinces = provinces.drop_duplicates()
        provinces['district_name'] = provinces.apply(lambda x: 'Otros distritos de la provincia de {}'.format(x['province_name']), axis=1)
        provinces['district_id'] = provinces['province_id'] + '00'
        provinces = provinces[['department_name', 'province_name', 'district_name', 'department_id', 'province_id', 'district_id']].copy()
        
        departments = df[['department_name', 'department_id']].copy()
        departments = departments.drop_duplicates()
        departments['province_name'] = departments.apply(lambda x: 'Otras provincias del departamento de {}'.format(x['department_name']), axis=1)
        departments['district_name'] = departments.apply(lambda x: 'Otros distritos del departamento de {}'.format(x['department_name']), axis=1)
        departments['province_id'] = departments['department_id'] + '00'
        departments['district_id'] = departments['province_id'] + '00'
        departments = departments[['department_name', 'province_name', 'district_name', 'department_id', 'province_id', 'district_id']].copy()
        
        df = df.append(provinces)
        df = df.append(departments)

        df['nation_id'] = 'per'
        df['nation_name'] = 'Perú'

In [113]:
ubigeo = df

In [114]:
nation_map = {k:v for (k,v) in zip(ubigeo["nation_name"], ubigeo["nation_id"])}
department_map = {k:v for (k,v) in zip(ubigeo["department_name"], ubigeo["department_id"])}
district_map = {k:v for (k,v) in zip(ubigeo["district_name"], ubigeo["district_id"])}
province_map = {k:v for (k,v) in zip(ubigeo["province_name"], ubigeo["province_id"])}

In [115]:
ARCHIVOS_LIST = {
    1 : "01. Puntos de Cultura_2020_DataPeru.xlsx",
    2 : "02. RCN_2020_PJ_DATAPERU.xlsx", 
    3 : "03.  EEC_2019_DATAPERU.xlsx",
    4 : "04. Agentes del libro_2020_PNYPJ_DATAPERU.xlsx"
}

In [116]:
data = {}
datasets = {}
for i in range (1, 5):
    data[i] = pd.ExcelFile("../../../datasets/20201018/05. Socios Estratégicos - Ministerio de Cultura (18-10-2020)/{}".format(ARCHIVOS_LIST[i]))
    sheet = data[i].sheet_names[1]
    datasets[i] = pd.read_excel(data[i], data[i].sheet_names[1])

In [117]:
datasets[1].columns

Index(['N', 'Código', 'Nombre', 'Región', 'Provincia', 'Distrito', 'Web/blog',
       'Facebook (Fan page y/o perfil)', 'Twitter', 'YouTube', 'Instagram',
       'Otra red social', 'Tipo de organización', 'Año de fundación',
       '¿La organización está inscrita en SUNARP?', 'N° Partida Registral',
       'N° RUC', 'Cantidad de miembros', 'Actividad realizada N° 1',
       'Actividad realizada N° 2', 'Manifestación artística cultural N° 1',
       'Manifestación artística cultural N° 2',
       'Manifestación artística cultural N° 3', 'N° Resolución Directoral',
       'Fecha de Reconocimiento', 'Nº de Registro '],
      dtype='object')

In [118]:
pivot = datasets[1][['Código','Nombre','Distrito','¿La organización está inscrita en SUNARP?','Actividad realizada N° 1',
       'Actividad realizada N° 2', 'Manifestación artística cultural N° 1',
       'Manifestación artística cultural N° 2',
       'Manifestación artística cultural N° 3']]

In [119]:
pivot

,Código,Nombre,Distrito,¿La organización está inscrita en SUNARP?,Actividad realizada N° 1,Actividad realizada N° 2,Manifestación artística cultural N° 1,Manifestación artística cultural N° 2,Manifestación artística cultural N° 3
0,S0004,Asociación de Grupo Teatral de Mala - GRUTEMA,Mala,SI,Actividades constantes que desde el arte y la ...,Actividades constantes que fomentan la apropia...,Animación/mediación sociocultural,"Artes escénicas - circo, teatro",Música
1,S0001,Creeando Arte con Propósito,Ventanilla,SI,Actividades que aportan a la recuperación y/o ...,Propuestas de desarrollo económico sostenible ...,Animación/mediación sociocultural,Artes escénicas - danza,"Celebración de festividades, fiestas tradicion..."
2,S0002,Asociación Cultural Dreams Imagination,Trujillo,SI,Actividades que aportan a la recuperación y/o ...,Actividades constantes que desde el arte y la ...,"Artes escénicas - circo, teatro","Audiovisuales - cine, video y/u otros",Música
3,S0003,Agenda CIX,Chiclayo,NO,Actividades que aportan a la recuperación y/o ...,Actividades que fortalecen las experiencias de...,Animación/mediación sociocultural,"Celebración de festividades, fiestas tradicion...",Música
4,S0007,Asociación Cultural Apoyo Mutuo,Miraflores,SI,Actividades que aportan a la recuperación y/o ...,Otras actividades relacionadas al arte para la...,Animación/mediación sociocultural,"Audiovisuales - cine, video y/u otros","Comunicaciones (periodismo, radio, podcast y/u..."
...,...,...,...,...,...,...,...,...,...
387,S0358,AYLLU GUTA,Trujillo,NO,Actividades que aportan a la recuperación y/o ...,Actividades constantes que fomentan la apropia...,Animación/mediación sociocultural,Artes visuales y artesanía,Otro: Recuperación de espacios públicos a trav...
388,S0357,ASOCIACIÓN CULTURAL LAS SHICRAS,San Martín de Porres,SI,Propuestas de desarrollo económico sostenible ...,"Actividades que aportan al rescate, protección...",Artes escénicas- danza,Artes visuales y artesanía,NaN
389,S0356,BALLET FOLKLÓRICO HUELLAS DE MI TIERRA,Villa El Salvador,NO,Actividades constantes que desde el arte y la ...,Actividades que promueven el respeto a la dive...,Artes escénicas- danza,"Celebración de festividades, fiestas tradicion...",Música
390,S0347,Danzas y Marinera Mi Perú,Chepén,NO,Actividades constantes que desde el arte y la ...,Actividades que promueven el respeto a la dive...,Animación/mediación sociocultural,Artes escénicas- danza,"Celebración de festividades, fiestas tradicion..."


In [120]:
pivot["Código"] = pivot["Código"].str.strip()

In [121]:
pivot["Actividad realizada N° 1"] = pivot["Actividad realizada N° 1"].str.strip()

In [122]:
pivot["Actividad realizada N° 2"] = pivot["Actividad realizada N° 2"].str.strip()

In [123]:
pivot["Actividad realizada N° 2"] = pivot["Actividad realizada N° 2"].str.strip()

In [124]:
pivot["Manifestación artística cultural N° 1"] = pivot["Manifestación artística cultural N° 1"].str.strip()

In [125]:
pivot["Manifestación artística cultural N° 2"] = pivot["Manifestación artística cultural N° 2"].str.strip()

In [126]:
pivot["Manifestación artística cultural N° 3"] = pivot["Manifestación artística cultural N° 3"].str.strip()

In [127]:
pivot  = pivot.rename(columns = {"Código" : "codigo_asociacion", "¿La organización está inscrita en SUNARP?" : "inscrita_sunarp_id", "Actividad realizada N° 1" : "actividad_n_1_id",  "Actividad realizada N° 2" : "actividad_n_2_id", "Manifestación artística cultural N° 1": "manifestacion_n_1_id", "Manifestación artística cultural N° 2": "manifestacion_n_2_id", "Manifestación artística cultural N° 3": "manifestacion_n_3_id","Distrito" : "district_name","Nombre" : "asociacion_name"})

In [128]:
pivot['cantidad_asociacion'] = 1

In [129]:
ubigeo['district_name'].unique()

array(['Chachapoyas', 'Asunción', 'Balsas', ...,
       'Otros distritos del departamento de Tumbes',
       'Otros distritos del departamento de Ucayali',
       'Otros distritos del departamento de Otro'], dtype=object)

In [130]:
pivot['district_name'].unique()

array(['Mala', 'Ventanilla', 'Trujillo', 'Chiclayo', 'Miraflores',
       'Callao', 'Chincha Alta', 'Jesús María', 'Villa el Salvador',
       'Tarapoto', 'Villa El Salvador', 'Chepén ', 'Santiago de Surco',
       'Huanchaco', 'Banda de Shilcayo', 'Moyobamba', 'Chorrillos',
       'Tacna', 'San Isidro', 'San Miguel', 'El Agustino', 'Ayaviri',
       'Coronel Gregorio Albarracín Lanchipa', 'Mariano Melgar',
       'La Victoria ', 'Huancayo', 'Pueblo Libre',
       'San Juan de Lurigancho', 'Alto Selva Alegre', 'El Carmen',
       'Moquegua', 'San Bartolomé', 'Andahuaylas', 'Lurigancho',
       'Carabayllo', 'San Jerónimo', 'Villa María del Triunfo', 'Tumbes',
       'Rímac', 'La Esperanza ', 'Moyobamba ', 'Comas', 'Barranco',
       'San Martin de Porres', 'Ate', 'Catacaos', 'Chazuta', 'Uchumayo',
       'Lima', 'Moro', 'Ayavirí', 'Huaraz', 'Morales', 'La Victoria',
       'Ventanilla ', 'Tambopata', 'Puno', 'Ferreñafe ', 'Chancay',
       'Juanjui', 'La Perla ', 'La Perla', 'Rioja', '

In [131]:
[x for x in pivot['district_name'].unique() if x not in ubigeo['district_name'].unique()]

['Villa el Salvador',
 'Chepén ',
 'Banda de Shilcayo',
 'La Victoria ',
 'La Esperanza ',
 'Moyobamba ',
 'San Martin de Porres',
 'Ayavirí',
 'Ventanilla ',
 'Ferreñafe ',
 'Juanjui',
 'La Perla ',
 'MIraflores',
 'Villa María del Triunfo ',
 '26 de Octubre',
 'Independencia ',
 'Chiclayo ',
 'Veintiseis de octubre',
 'Ate ',
 'Cayma ',
 'Castilla ',
 'Gregorio Albarracín',
 'Chepén',
 'Ica ',
 'Callao ',
 'Huancayo ',
 'Pillo Marca',
 'Callería',
 'Nuevo chimbote',
 'La banda de Silchayo',
 'Ate Vitarte',
 'Pardo Miguel Naranjos',
 'Huacho Mariscal Castilla N°400 (Interior 3) Huacho',
 'Huánuco',
 'Juliaca ',
 'Jesus María',
 'Ascención',
 'LIMA',
 'San Sebastián',
 'José Luis Bustamante y Rivero']

In [132]:
DISTRICT_REPLACE_1 = {
'Villa el Salvador' : "Villa El Salvador",
 'Chepén ' : "Chepen",
 'Banda de Shilcayo' : "La Banda de Shilcayo",
 'La Victoria ' : "La Victoria",
 'La Esperanza ' : "La Esperanza",
 'Moyobamba ' : "Moyobamba",
 'San Martin de Porres' : "San Martín de Porres",
 'Ayavirí' : "Ayaviri",
 'Ventanilla ' : "Ventanilla",
 'Ferreñafe ' : "Ferreñafe",
 'Juanjui': "Juanjuí",
 'La Perla ': "La Perla",
 'MIraflores': "Miraflores",
 'Villa María del Triunfo ': "Villa María del Triunfo",
 '26 de Octubre': "Veintiseis de Octubre",
 'Independencia ': "Independencia",
 'Chiclayo ': "Chiclayo",
 'Veintiseis de octubre': "Veintiseis de Octubre",
 'Ate ': "Ate",
 'Cayma ': "Cayma",
 'Castilla ': "Castilla",
 'Gregorio Albarracín': "Coronel Gregorio Albarracín Lanchipa",
 'Chepén' : "Chepen",
 'Ica ' : "Ica",
 'Callao ' : "Callao",
 'Huancayo ' : "Huancayo",
 'Pillo Marca' : "Pillco Marca",
 'Callería' : "Calleria",
 'Nuevo chimbote' : "Nuevo Chimbote",
 'La banda de Silchayo' : "La Banda de Shilcayo",
 'Ate Vitarte' : "Ate",
 'Pardo Miguel Naranjos' : "Pardo Miguel",
 'Huacho Mariscal Castilla N°400 (Interior 3) Huacho' : "Huacho",
 'Huánuco' : "Huanuco",
 'Juliaca ' : "Juliaca",
 'Jesus María' : "Jesús María",
 'Ascención' : "Asunción",
 'LIMA' : "Lima",
 'San Sebastián' : "San Sebastian",
 'José Luis Bustamante y Rivero' : "José Luis Bustamante Y Rivero",

}

In [133]:
pivot['district_name'] = pivot['district_name'].replace(DISTRICT_REPLACE_1)

In [134]:
for column in ['manifestacion_n_1_id','manifestacion_n_2_id', 'manifestacion_n_3_id']:
    pivot[column] = pivot[column].str.replace('Otro:', '' )
    pivot[column] = pivot[column].str.replace('Otros:', '').str.strip()

In [135]:
pivot['manifestacion_n_2_id'] = pivot['manifestacion_n_2_id'].str.replace('Otro :: desarrollo de los elementos del HIP HOP', 'Desarrollo de los elementos del HIP HOP')

In [136]:
asociacion_dim = dict(zip(pivot['codigo_asociacion'], pivot['asociacion_name']))

In [137]:
actividad_n_1_dim = dict(zip(pivot['actividad_n_1_id'].dropna().unique(), range(1, len(pivot['actividad_n_1_id'].dropna().unique()) + 1 )))
pivot['actividad_n_1_id'].replace(actividad_n_1_dim, inplace=True)

In [138]:
actividad_n_1_dim

{'Actividades constantes que desde el arte y la cultura atienden a poblaciones en condiciones de vulnerabilidad social': 1,
 'Actividades que aportan a la recuperación y/o el fortalecimiento del tejido social y la vida comunitaria desde procesos artísticos, culturales, de comunicación o educación': 2,
 'Actividades que aportan al rescate, protección, visibilización o revitalización de los saberes ancestrales, tradiciones, identidad cultural y memoria de pueblos originarios, campesinos, indígenas, rurales y poblaciones afroperuanas': 3,
 'Otras actividades relacionadas al arte para la transformación social, desarrolladas en el marco de la Ley N° 30487, Ley de promoción de los puntos de cultura': 4,
 'Propuestas de desarrollo económico sostenible a partir de los activos culturales de una comunidad, en respeto con la identidad cultural local y su entorno': 5,
 'Actividades que promueven el respeto a la diversidad de identidades y reconocimiento de derechos de poblaciones y/o colectivos qu

In [139]:
actividad_n_2_dim = dict(zip(pivot['actividad_n_2_id'].dropna().unique(), range(1, len(pivot['actividad_n_2_id'].dropna().unique()) + 1 )))
pivot['actividad_n_2_id'].replace(actividad_n_2_dim, inplace=True)

In [140]:
actividad_n_2_dim

{'Actividades constantes que fomentan la apropiación social del espacio público y patrimonio cultural desde acciones artísticas y culturales': 1,
 'Propuestas de desarrollo económico sostenible a partir de los activos culturales de una comunidad, en respeto con la identidad cultural local y su entorno': 2,
 'Actividades constantes que desde el arte y la cultura atienden a poblaciones en condiciones de vulnerabilidad social': 3,
 'Actividades que fortalecen las experiencias de trabajo en red y acción colaborativa que apuntan al empoderamiento de movimientos ciudadanos, incidencia y co-construcción de políticas públicas culturales que aporten al desarrollo social': 4,
 'Otras actividades relacionadas al arte para la transformación social, desarrollados en el marco de la Ley N° 30487, Ley de promoción de los puntos de cultura': 5,
 'Actividades que aportan al rescate, protección, visibilización o revitalización de los saberes ancestrales, tradiciones, identidad cultural y memoria de puebl

In [141]:
manifestacion_n_1_dim = dict(zip(pivot['manifestacion_n_1_id'].dropna().unique(), range(1, len(pivot['manifestacion_n_1_id'].dropna().unique()) + 1 )))
pivot['manifestacion_n_1_id'].replace(manifestacion_n_1_dim, inplace=True)

In [142]:
manifestacion_n_2_dim = dict(zip(pivot['manifestacion_n_2_id'].dropna().unique(), range(1, len(pivot['manifestacion_n_2_id'].dropna().unique()) + 1 )))
pivot['manifestacion_n_2_id'].replace(manifestacion_n_2_dim, inplace=True)

In [143]:
manifestacion_n_3_dim = dict(zip(pivot['manifestacion_n_3_id'].dropna().unique(), range(1, len(pivot['manifestacion_n_3_id'].dropna().unique()) + 1 )))
pivot['manifestacion_n_3_id'].replace(manifestacion_n_3_dim, inplace=True)

In [144]:
inscrita_sunarp_dim = dict(zip(pivot['inscrita_sunarp_id'].dropna().unique(), range(1, len(pivot['inscrita_sunarp_id'].dropna().unique()) + 1 )))
pivot['inscrita_sunarp_id'].replace(inscrita_sunarp_dim, inplace=True)

In [145]:
inscrita_sunarp_dim

{'SI': 1, 'NO': 2, 'No': 3}

In [146]:
manifestacion_n_3_dim

{'Música': 1,
 'Celebración de festividades, fiestas tradicionales y/o rituales': 2,
 'Comunicaciones (periodismo, radio, podcast y/u otra)': 3,
 'Tics, nuevos medios y/o videojuegos': 4,
 'Deporte, recreación y/o juego': 5,
 'Libro, lectura y/o escritura creativa': 6,
 'Lengua de señas peruana y cultura sorda': 7,
 'Artes visuales y artesanía': 8,
 'Muralización': 9,
 'Tradición oral propia de pueblos indígenas u originarios': 10,
 'Investigación, libros, tradición, etc': 11,
 'Comida / Gastronomía': 12,
 'Enseñanza del inglés mediante el Teatro': 13,
 'Artes escénicas - danza': 14,
 'Gestión Museografia': 15,
 'Formación de orientadores': 16,
 'Medicina tradicional': 17,
 'Capacitación para nuevos actores': 18,
 'Arte Comunitario': 19,
 'Performance - ópera hablada': 20,
 'Foros, talleres, capacitaciones, workshops, clases maestras y charlas': 21,
 'Audiovisuales - cine, video y/u otros': 22,
 'Fortalecimiento de capacidades para organizaicones y agentes sociales y culturales, en pro

In [147]:
df1 = pivot[['codigo_asociacion','district_name'
       'inscrita_sunarp_id', 'actividad_n_1_id', 'actividad_n_2_id',
       'manifestacion_n_1_id', 'manifestacion_n_2_id', 'manifestacion_n_3_id',
       'cantidad_asociacion']]

KeyError: "['district_nameinscrita_sunarp_id'] not in index"

In [148]:
df1

NameError: name 'df1' is not defined

In [134]:
##### Registro cinematografico nacional

In [304]:
## solo hay 1268 registros

datasets[2] = datasets[2][0:1267]

In [305]:
beta = datasets[2][['AÑO INSCRIPCIÓN',
       'TIPO DE CONSTITUCIÓN', 'RAZON SOCIAL ',
       'Actividad_1', 'Actividad_2', 'Actividad_3', 'Actividad_4',
       'DISTRITO']]

In [306]:
beta.columns

Index(['AÑO INSCRIPCIÓN', 'TIPO DE CONSTITUCIÓN', 'RAZON SOCIAL ',
       'Actividad_1', 'Actividad_2', 'Actividad_3', 'Actividad_4', 'DISTRITO'],
      dtype='object')

In [307]:
beta = beta.rename(columns={'AÑO INSCRIPCIÓN' : "anio_inscripcion","TIPO DE CONSTITUCIÓN" : "tipo_constitucion_id","RAZON SOCIAL " : "razon_social_id", "Actividad_1" : "actividad_1_id","Actividad_2":"actividad_2_id", "Actividad_3":"actividad_3_id", "Actividad_4":"actividad_4_id","DISTRITO":"district_name"})

In [308]:
beta['cantidad_org'] = 1

In [309]:
[x for x in beta['district_name'].unique() if x not in ubigeo['district_name'].unique()]

['Cerro colorado',
 'Huáchac',
 'Callería',
 '-',
 'Raimondi',
 'Huánuco',
 'Villa el Salvador',
 'Samuel Pastor ',
 'Cineguilla',
 'José Luis Bustamante y Rivero',
 'Ate ',
 'San Sebastián',
 'San Borja ',
 'Lurín',
 'Huangáscar',
 'Veintiséis de Octubre',
 'Rimac',
 'Santa anita',
 'La Victoria ',
 'San borja ',
 'Santa anita ',
 'San Vicente',
 'San Miguel ',
 'Miraflores ',
 'Comas ',
 'Barranco ',
 'Cusco ',
 'Santiago  ',
 'Santa María Del Mar',
 'La banda de Shilcayo',
 'Juliaca ',
 'Ayacucho ',
 'Cercado',
 'Juliana',
 'Cercado de Lima',
 nan]

In [310]:
DISTRICT_REPLACE_2 = {
'Cerro colorado' : "Cerro Colorado",
 'Huáchac' : "Huachac",
 'Calleria ' : "Calleria",
 '-' : np.nan,
 'Raimondi' : "Raymondi",
 'Huánuco' : "Huanuco",
 'Villa el Salvador' : "Villa El Salvador",
 'Samuel Pastor ' : "Samuel Pastor",
 'Cineguilla' : "Cieneguilla",
 'José Luis Bustamante y Rivero' : "José Luis Bustamante Y Rivero",
 'Ate ' : "Ate",
 'San Sebastián' : "San Sebastian",
 'San Borja ' : "San Borja",
 'Lurín' : "Lurin",
 'Huangáscar' : "Huangascar",
 'Veintiséis de Octubre' : "Veintiseis de Octubre",
 'Rimac' : "Rímac",
 'Santa anita' : "Santa Anita",
 'La Victoria ' : "La Victoria",
 'San borja ' : "San Borja",
 'Santa anita ' : "Santa Anita",
 'San Vicente' : "San Vicente de Cañete",
 'San Miguel ' : "San Miguel",
 'Miraflores ' : "Miraflores",
 'Comas ' : "Comas",
 'Barranco ' : "Barranco",
 'Cusco ' : "Cusco",
 'Santiago  ' : "Santiago",
 'Santa María Del Mar' : "Santa María del Mar",
 'La banda de Shilcayo' : "La Banda de Shilcayo",
 'Juliaca ' : "Juliaca",
 'Ayacucho ' : "Ayacucho",
 'Cercado' : "Lima",
 'Juliana' : "Juliaca",
 'Cercado de Lima' : "Lima",


}

In [311]:
beta['district_name'] = beta['district_name'].replace(DISTRICT_REPLACE_2)

In [312]:
beta['tipo_constitucion_id'] = beta['tipo_constitucion_id'].replace("Empresa ", "Empresa")

In [313]:
beta = beta[beta['district_name'].notna()]

In [314]:
beta = beta[beta['anio_inscripcion'] != "ND"]

In [315]:
beta

,anio_inscripcion,tipo_constitucion_id,razon_social_id,actividad_1_id,actividad_2_id,actividad_3_id,actividad_4_id,district_name,cantidad_org
234,2018,EMPRESA,Ave Fantasma Productora Cinematográfica E.I.R.L.,Productora,NaN,NaN,NaN,Lima,1
235,2004,EMPRESA,Aguardiente Producciones SAC,Productora,NaN,NaN,NaN,Jesús María,1
236,2012,EMPRESA,Abril Producciones SAC,Productora,NaN,NaN,NaN,Miraflores,1
237,2012,EMPRESA,Aldea Digital Comunicaciones SAC,Productora,NaN,NaN,NaN,Villa El Salvador,1
238,2012,EMPRESA,Alelo Films SAC,Productora,NaN,NaN,NaN,Surquillo,1
...,...,...,...,...,...,...,...,...,...
1262,2020,EMPRESA,Colmena Estudia E.I.R.L.,Productora,NaN,NaN,NaN,Villa María del Triunfo,1
1263,2020,EMPRESA,Tambo Productions E.I.R.L.,Productora,NaN,NaN,NaN,Barranco,1
1264,2020,EMPRESA,Peruana de Realización Audiovisual,Productora,NaN,NaN,NaN,Pueblo Libre,1
1265,2020,EMPRESA,Vuyana E.I.R.L.,Productora,NaN,NaN,NaN,La Esperanza,1


In [316]:
  for column in ['tipo_constitucion_id']:

            beta[column] = beta[column].str.strip()

In [317]:
#tipo_constitucion_dim = dict(zip(beta['tipo_constitucion_id'].dropna().unique(), range(1, len(beta['tipo_constitucion_id']#.dropna().unique()) + 1 )))
#beta['tipo_constitucion_id'].replace(tipo_constitucion_dim, inplace=True)
#
#razon_social_dim = dict(zip(beta['razon_social_id'].dropna().unique(), range(1, len(beta['razon_social_id'].dropna()#.unique()) + 1 )))
#beta['razon_social_id'].replace(razon_social_dim, inplace=True)
#
#actividad_1_dim = dict(zip(beta['actividad_1_id'].dropna().unique(), range(1, len(beta['actividad_1_id'].dropna().unique()#) + 1 )))
#beta['actividad_1_id'].replace(actividad_1_dim, inplace=True)
#
#actividad_2_dim = dict(zip(beta['actividad_2_id'].dropna().unique(), range(1, len(beta['actividad_2_id'].dropna().unique()#) + 1 )))
#beta['actividad_2_id'].replace(actividad_2_dim, inplace=True)
#
#actividad_3_dim = dict(zip(beta['actividad_3_id'].dropna().unique(), range(1, len(beta['actividad_3_id'].dropna().unique()#) + 1 )))
#beta['actividad_3_id'].replace(actividad_3_dim, inplace=True)

In [318]:
tipo_constitucion_dim

{'EMPRESA': 1, 'ASOCIACIÓN': 2}

In [319]:
razon_social_dim = dict(zip(beta['razon_social_id'].dropna().unique(), range(1, len(beta['razon_social_id'].dropna().unique()) + 1 )))
beta['razon_social_id'].replace(razon_social_dim, inplace=True)

In [320]:
actividad_1_dim = dict(zip(beta['actividad_1_id'].dropna().unique(), range(1, len(beta['actividad_1_id'].dropna().unique()) + 1 )))
beta['actividad_1_id'].replace(actividad_1_dim, inplace=True)


In [321]:
actividad_1_dim

{'Productora': 1,
 'Servicios conexos': 2,
 'Exhibidora': 3,
 'Distribuidora': 4,
 'Formación': 5}

In [322]:
actividad_2_dim = dict(zip(beta['actividad_2_id'].dropna().unique(), range(1, len(beta['actividad_2_id'].dropna().unique()) + 1 )))
beta['actividad_2_id'].replace(actividad_2_dim, inplace=True)
actividad_2_dim

{'Servicios conexos': 1, 'Distribuidora': 2, 'Exhibidora': 3}

In [323]:
actividad_3_dim = dict(zip(beta['actividad_3_id'].dropna().unique(), range(1, len(beta['actividad_3_id'].dropna().unique()) + 1 )))
beta['actividad_3_id'].replace(actividad_3_dim, inplace=True)
actividad_3_dim

{'Distribuidora': 1}

In [324]:
actividad_4_dim = dict(zip(beta['actividad_4_id'].dropna().unique(), range(1, len(beta['actividad_4_id'].dropna().unique()) + 1 )))
beta['actividad_4_id'].replace(actividad_4_dim, inplace=True)
actividad_4_dim

{'Servicios conexos': 1}

In [325]:
beta

,anio_inscripcion,tipo_constitucion_id,razon_social_id,actividad_1_id,actividad_2_id,actividad_3_id,actividad_4_id,district_name,cantidad_org
234,2018,EMPRESA,1,1,NaN,NaN,NaN,Lima,1
235,2004,EMPRESA,2,1,NaN,NaN,NaN,Jesús María,1
236,2012,EMPRESA,3,1,NaN,NaN,NaN,Miraflores,1
237,2012,EMPRESA,4,1,NaN,NaN,NaN,Villa El Salvador,1
238,2012,EMPRESA,5,1,NaN,NaN,NaN,Surquillo,1
...,...,...,...,...,...,...,...,...,...
1262,2020,EMPRESA,1025,1,NaN,NaN,NaN,Villa María del Triunfo,1
1263,2020,EMPRESA,1026,1,NaN,NaN,NaN,Barranco,1
1264,2020,EMPRESA,1027,1,NaN,NaN,NaN,Pueblo Libre,1
1265,2020,EMPRESA,1028,1,NaN,NaN,NaN,La Esperanza,1


In [326]:
datasets[3].head()

,Nro,Dirección,EEC,Fase de cadena de valor_E,Nro de proyecto,Expediente,Nombre del proyecto,Tipo de Postulante,Postulante,N° RUC,DNI_Postulante,País,Región,Provincia,Distrito,Estado revisado
0,1.0,DAFO,C01. Proyectos de Largometraje de Ficción,3. Producción,Proyecto 0001,CFN-2-P-001-19,Vallejo Inmortal,Persona Jurídica,MALVADO FILMS E.I.R.L.,NaN,NaN,PERÚ,LIMA,LIMA,MIRAFLORES,No beneficiario
1,2.0,DAFO,C01. Proyectos de Largometraje de Ficción,3. Producción,Proyecto 0002,CFN-2-P-002-19,Mamita,Persona Jurídica,ESFENA S.A.C.,NaN,NaN,PERÚ,LIMA,LIMA,MIRAFLORES,No beneficiario
2,3.0,DAFO,C01. Proyectos de Largometraje de Ficción,3. Producción,Proyecto 0003,CFN-2-P-004-19,Dos Estrellas,Persona Jurídica,GALLINAZOS COMUNICACIONES S.A.C.,NaN,NaN,PERÚ,LIMA,LIMA,SANTIAGO DE SURCO,No beneficiario
3,4.0,DAFO,C01. Proyectos de Largometraje de Ficción,3. Producción,Proyecto 0004,CFN-2-P-005-19,CERO,Persona Jurídica,CAMAL ESTUDIO CREATIVO SOCIEDAD COMERCIAL DE R...,NaN,NaN,PERÚ,CUSCO,CUSCO,SAN JERONIMO,No beneficiario
4,5.0,DAFO,C01. Proyectos de Largometraje de Ficción,3. Producción,Proyecto 0005,CFN-2-P-006-19,"CENEPA 1995, El Legado.",Persona Jurídica,EMPRENDECINE PRODUCCIONES S.A.C.,NaN,NaN,PERÚ,LIMA,LIMA,SAN LUIS,No beneficiario


In [327]:
gamma = datasets[3][['EEC', 'Fase de cadena de valor_E',
       'Nombre del proyecto',
       'Tipo de Postulante', 'Postulante','País',
       'Distrito', 'Estado revisado']]

In [328]:
gamma = gamma.rename(columns={'EEC' : 'estimulo_economico_id', 'Fase de cadena de valor_E' : "fase_cadena_valor_id", "Nombre del proyecto" : "nombre_proyecto_id", "Tipo de Postulante" :  "tipo_postulante_id","Postulante" : "postulante_id", "País": "nation_id", "Distrito" : "district_id", 'Estado revisado' :  "estado_id"})

In [329]:
gamma = gamma[gamma['district_id'].notna()]

In [330]:
gamma['estimulo_economico_id'] = gamma['estimulo_economico_id'].str[4:].str.strip()

In [331]:
gamma['fase_cadena_valor_id'] = gamma['fase_cadena_valor_id'].str[2:].str.strip()

In [332]:
gamma["tipo_postulante_id"] = gamma["tipo_postulante_id"].replace({'Persona natural': 'Persona Natural'})

In [333]:
gamma['nation_id'] = gamma['nation_id'].str.capitalize()

In [334]:
gamma['district_id'] = gamma['district_id'].str.strip()
gamma['district_id'] = gamma['district_id'].str.title()

In [335]:
[x for x in gamma['district_id'].unique() if x not in ubigeo['district_name'].unique()]

['Santiago De Surco',
 'San Jeronimo',
 'Villa Maria Del Triunfo',
 'Jesus Maria',
 'Magdalena Del Mar',
 'San Juan De Lurigancho',
 'Jose Luis Bustamante Y Rivero',
 'San Martin De Porres',
 'N/D',
 'Belen',
 'San Vicente De Cañete',
 'San Juan De Miraflores',
 'Rimac',
 'Simon Bolivar',
 'Ancon',
 'Santa Maria Del Mar',
 'Los Baños Del Inca',
 'Jesus Nazareno',
 'Veintiseis De Octubre',
 'Azangaro',
 'Jose Leonardo Ortiz',
 'Coronel Gregorio Albarracin Lanchipa',
 'No Aplica']

In [336]:
DISTRICT_REPLACE_3 = {
 'Santiago De Surco' : "Santiago de Surco",
 'San Jeronimo' : "San Jerónimo",
 'Villa Maria Del Triunfo' : "Villa María del Triunfo",
 'Jesus Maria' : "Jesús María",
 'Magdalena Del Mar' : "Magdalena del Mar",
 'San Juan De Lurigancho' : "San Juan de Lurigancho",
 'Jose Luis Bustamante Y Rivero' : "José Luis Bustamante Y Rivero",
 'San Martin De Porres' : "San Martín de Porres",
 'N/D' : "Otro",
 'Belen' : "Belén",
 'San Vicente De Cañete' : "San Vicente de Cañete",
 'San Juan De Miraflores' : "San Juan de Miraflores",
 'Rimac' : "Rímac",
 'Simon Bolivar' : "Simon Bolívar",
 'Ancon' : "Ancón",
 'Santa Maria Del Mar' : "Santa María del Mar",
 'Los Baños Del Inca' : "Los Baños del Inca",
 'Jesus Nazareno' : "Jesús Nazareno",
 'Veintiseis De Octubre' : "Veintiseis de Octubre",
 'Azangaro' : "Azángaro",
 'Jose Leonardo Ortiz' : "José Leonardo Ortiz",
 'Coronel Gregorio Albarracin Lanchipa' : "Coronel Gregorio Albarracín Lanchipa",
 'No Aplica' : "Otro",

}

In [337]:
gamma['district_id'] = gamma['district_id'].replace(DISTRICT_REPLACE_3)

In [338]:
gamma.tipo_postulante_id = gamma.tipo_postulante_id.fillna("Otro")

In [339]:
gamma.tipo_postulante_id.unique()

array(['Persona Jurídica', 'Persona Natural', 'Otro'], dtype=object)

In [340]:
estimulo_economico_dim = dict(zip(gamma['estimulo_economico_id'].dropna().unique(), range(1, len(gamma['estimulo_economico_id'].dropna().unique()) + 1 )))
gamma['estimulo_economico_id'].replace(estimulo_economico_dim, inplace=True)
estimulo_economico_dim

{'Proyectos de Largometraje de Ficción': 1,
 'Proyectos de Gestión Cultural para el Audiovisual': 2,
 'Cortometrajes': 3,
 'Estímulo a la Distribución Cinematográfica': 4,
 'Estímulo a la Promoción Internacional': 5,
 'Proyectos de Cortometraje': 6,
 'Desarrollo de Proyectos de Largometraje': 7,
 'Proyectos de Documental Formato Largo': 8,
 'Obras Experimentales': 9,
 'Proyectos de Distribución de Largometraje': 10,
 'Proyectos de Preproducción de Largometraje de Animación': 11,
 'Circulación Internacional para las Artes-Primera llamada': 12,
 'Circulación Internacional para las Artes-Segunda llamada': 13,
 'Escritura de Literatura Infantil y Juvenil': 14,
 'Formación Audiovisual': 15,
 'Proyectos de Largometraje de Ficción Exclusivo para las Regiones del País (excepto Lima Metropolitana y Callao)': 16,
 'Proyectos de Largometraje de Ficción (Estímulo Alternativo)': 17,
 'Coproducciones Minoritarias': 18,
 'Proyectos de Preservación Audiovisual': 19,
 'Proyectos de Gestión de Salas de 

In [341]:
fase_cadena_valor_dim = dict(zip(gamma['fase_cadena_valor_id'].dropna().unique(), range(1, len(gamma['fase_cadena_valor_id'].dropna().unique()) + 1 )))
gamma['fase_cadena_valor_id'].replace(fase_cadena_valor_dim, inplace=True)
fase_cadena_valor_dim

{'Producción': 1, 'Acceso': 2, 'Creación': 3, 'Circulación': 4, 'Formación': 5}

In [342]:
nombre_proyecto_dim = dict(zip(gamma['nombre_proyecto_id'].dropna().unique(), range(1, len(gamma['nombre_proyecto_id'].dropna().unique()) + 1 )))
gamma['nombre_proyecto_id'].replace(nombre_proyecto_dim, inplace=True)


In [343]:
tipo_postulante_dim = dict(zip(gamma['tipo_postulante_id'].dropna().unique(), range(1, len(gamma['tipo_postulante_id'].dropna().unique()) + 1 )))
gamma['tipo_postulante_id'].replace(tipo_postulante_dim, inplace=True)
tipo_postulante_dim

{'Persona Jurídica': 1, 'Persona Natural': 2, 'Otro': 3}

In [344]:
postulante_dim = dict(zip(gamma['postulante_id'].dropna().unique(), range(1, len(gamma['postulante_id'].dropna().unique()) + 1 )))
gamma['postulante_id'].replace(postulante_dim, inplace=True)
#postulante_dim

In [345]:
estado_dim = dict(zip(gamma['estado_id'].dropna().unique(), range(1, len(gamma['estado_id'].dropna().unique()) + 1 )))
gamma['estado_id'].replace(estado_dim, inplace=True)
estado_dim

{'No beneficiario': 1, 'Beneficiario': 2, 'No evaluado': 3}

In [346]:
gamma

,estimulo_economico_id,fase_cadena_valor_id,nombre_proyecto_id,tipo_postulante_id,postulante_id,nation_id,district_id,estado_id
0,1,1,1,1,1,Perú,Miraflores,1
1,1,1,2,1,2,Perú,Miraflores,1
2,1,1,3,1,3,Perú,Santiago de Surco,1
3,1,1,4,1,4,Perú,San Jerónimo,1
4,1,1,5,1,5,Perú,San Luis,1
...,...,...,...,...,...,...,...,...
2237,5,4,2181,3,1613,Perú,Jesús María,1
2238,5,4,2182,3,1614,Perú,Miraflores,1
2239,5,4,144,3,1615,Perú,Miraflores,1
2240,5,4,2183,3,1616,Perú,Miraflores,1


In [347]:
gamma['cantidad_postulante'] = 1

In [348]:
datasets[4].columns =  datasets[4].iloc[3]

In [349]:
datasets[4] = datasets[4][4:]

In [350]:
datasets[4].columns

Index([                                                12,
                                              'Editorial',
                                                      nan,
                                                      nan,
                                                      nan,
                            'Cascahuesos Editores S.A.C.',
                                              20454870914,
                                               'Arequipa',
                                               'Arequipa',
                                                  'Cayma',
              'https://cascahuesoseditores.blogspot.com/',
       'https://www.facebook.com/cascahuesoseditores.pe/',
                                      '@cascahuesos_edit',
                                       '@EditCascahuesos',
                                                      nan,
                                                      nan],
      dtype='object', name=15)

In [351]:
delta = datasets[4][['Nombre y/o razón social de la organización','Actividad_1', 'Actividad_2', 'Actividad_3', 'Actividad_4','Distrito de la organización']]

KeyError: "None of [Index(['Nombre y/o razón social de la organización', 'Actividad_1',\n       'Actividad_2', 'Actividad_3', 'Actividad_4',\n       'Distrito de la organización'],\n      dtype='object', name=15)] are in the [columns]"

In [195]:
delta.rename(columns={'Nombre y/o razón social de la organización': "razon_social_id", "Actividad_1" : "actividad_1_id", "Actividad_2" : "actividad_2_id", "Actividad_3" : "actividad_3_id", "Actividad_4" : "actividad_4_id", "Distrito de la organización": "district_id"}, inplace=True)

In [196]:
[x for x in delta['district_id'].unique() if x not in ubigeo['district_name'].unique()]

['Cercado de Lima',
 'Carabayllo ',
 'Huánuco ',
 'Pueblo libre',
 'El Tambo ',
 'San Sebastián',
 'José Luis Bustamante y Rivero',
 'Trujillo ',
 'Villa el Salvador',
 'Huancayo ',
 'Sin información',
 'Talara',
 'Callería',
 'Ancohuayllo Uripa']

In [197]:
DISTRICT_REPLACE_4 = {
 'Cercado de Lima' : "Lima",
 'Carabayllo ' : "Carabayllo",
 'Huánuco ' : "Huanuco",
 'Pueblo libre' : "Pueblo Libre",
 'El Tambo ': "Huanuco",
 'San Sebastián': "Huanuco",
 'José Luis Bustamante y Rivero': "José Luis Bustamante Y Rivero",
 'Trujillo ': "Trujillo",
 'Villa el Salvador': "Villa El Salvador",
 'Huancayo ': "Huancayo",
 'Sin información': "Otros",
 'Talara': "Chiclayo ",
 'Callería': "Calleria",
 'Ancohuayllo Uripa': "Anco_Huallo"
}

In [198]:
delta['district_id'] = delta['district_id'].replace(DISTRICT_REPLACE_4)

In [199]:
delta.head()

3,razon_social_id,actividad_1_id,actividad_2_id,actividad_3_id,actividad_4_id,district_id
4,Un puñado de tierra que respira S.R.L. (Hanan ...,Editorial,NaN,NaN,NaN,Villa María del Triunfo
5,Editorial Cthulhu,Editorial,NaN,NaN,NaN,Lima
6,Pobre cartonera,Cartonera,Fanzine,Otros,NaN,Pueblo Libre
7,Gato Viejo Producción Editorial S.A.C.,Editorial,NaN,NaN,NaN,San Juan de Lurigancho
8,Vivirsinenterarse S.A.C,Editorial,NaN,NaN,NaN,Carabayllo


In [200]:
razon_social_dim = dict(zip(delta['razon_social_id'].dropna().unique(), range(1, len(delta['razon_social_id'].dropna().unique()) + 1 )))
delta['razon_social_id'].replace(razon_social_dim, inplace=True)
#razon_social_dim

In [201]:
delta['actividad_1_id'] = delta['actividad_1_id'].replace('cartonera', 'Cartonera')

In [202]:
actividad_1_dim = dict(zip(delta['actividad_1_id'].dropna().unique(), range(1, len(delta['actividad_1_id'].dropna().unique()) + 1 )))
delta['actividad_1_id'].replace(actividad_1_dim, inplace=True)
actividad_1_dim

{'Editorial': 1,
 'Cartonera': 2,
 'Librería': 3,
 'Distribuidora': 4,
 'Revista': 5,
 'Imprenta': 6,
 'Fondos Universitarios': 7,
 'Prensa': 8,
 'Autor-editor': 9,
 'Fanzine': 10}

In [203]:
actividad_2_dim = dict(zip(delta['actividad_2_id'].dropna().unique(), range(1, len(delta['actividad_2_id'].dropna().unique()) + 1 )))
delta['actividad_2_id'].replace(actividad_2_dim, inplace=True)
actividad_2_dim

{'Fanzine': 1,
 'Libro Electrónico': 2,
 'Imprenta': 3,
 'Editorial': 4,
 'Librería': 5,
 'Distribuidora': 6}

In [204]:
actividad_3_dim = dict(zip(delta['actividad_3_id'].dropna().unique(), range(1, len(delta['actividad_3_id'].dropna().unique()) + 1 )))
delta['actividad_3_id'].replace(actividad_3_dim, inplace=True)
actividad_3_dim

{'Otros': 1, 'Librería': 2, 'Distribuidora': 3}

In [205]:
actividad_4_dim = dict(zip(delta['actividad_4_id'].dropna().unique(), range(1, len(delta['actividad_4_id'].dropna().unique()) + 1 )))
delta['actividad_4_id'].replace(actividad_4_dim, inplace=True)
actividad_4_dim

{'Librería': 1}

In [206]:
delta['cantidad_asociacion'] = 1

In [207]:
delta

3,razon_social_id,actividad_1_id,actividad_2_id,actividad_3_id,actividad_4_id,district_id,cantidad_asociacion
4,1,1,NaN,NaN,NaN,Villa María del Triunfo,1
5,2,1,NaN,NaN,NaN,Lima,1
6,3,2,1.0,1.0,NaN,Pueblo Libre,1
7,4,1,NaN,NaN,NaN,San Juan de Lurigancho,1
8,5,1,NaN,NaN,NaN,Carabayllo,1
...,...,...,...,...,...,...,...
431,427,1,NaN,NaN,NaN,Lima,1
432,428,4,NaN,NaN,NaN,Barranco,1
433,429,3,NaN,NaN,NaN,Lima,1
434,430,3,NaN,NaN,NaN,Barranco,1
